## Objectives
We will be using `AlphaSimR` to  
* Simulate datasets to test statistical genetic analysis methods  
* Simulate whole breeding programs in view of optimizing them  

Example code to work with `AlphaSimR` to  
1. Create a population and get breeding values from `AlphaSimR` and from an
"experiment"  
2. Prepare a data structure for breeding records that will be useful for
simulation.  
3. Use this data structure in simple linear model analyses.

### Script setup  
Install packages

In [ ]:
#Loading libraries
req_packages<-c("AlphaSimR", "tidyverse")

for(i in c(1:length(req_packages))){
  if (!require(req_packages[i], character.only = TRUE)){
   install.packages(req_packages[i])
  }
}

### Script parameters  

In [ ]:
random_seed <- 45678
set.seed(random_seed)
nFounders <- 100
nChr <- 7
segSites <- 140
nQTL <- 100

## Genotypic value versus breeding value  
By default, the additive genetic variance among founders will be 1.
Here are a couple of `AlphaSimR` genetic models with non-additive gene action. `addTraitAD` gives an additive + dominance gene action. Each locus has its own dominance deviation and there is a variance of it among loci:  
$dd \sim N(meanDD, varDD)$  
`addTraitADE` adds pairwise additive by additive epistasis.  

In [ ]:
# Create haplotypes for founder population of outbred individuals
founderHaps <- AlphaSimR::runMacs(nInd=nFounders, nChr=nChr, segSites=segSites)
# Setup the genotype to phenotype mapping
SP <- AlphaSimR::SimParam$new(founderHaps)
meanDD <- 0.4
varDD <- 0.04

# Additive and dominance model
SP$addTraitAD(nQtlPerChr=nQTL, meanDD=meanDD, varDD=varDD)

# Create a new population of founders
founders <- AlphaSimR::newPop(founderHaps, simParam=SP)

# Compare genotypic and breeding values
corBVGV <- cor(AlphaSimR::gv(founders), AlphaSimR::bv(founders)) %>% round(3)
plot(AlphaSimR::gv(founders), AlphaSimR::bv(founders), pch=16,
     xlab="Genotypic value", ylab="Breeding value",
     main=paste0("BV vs GV: AD model, cor=", corBVGV),
     cex.axis=1.3, cex.lab=1.3)

# Additive dominance and epistasis model
# "the relative value of additive-by-additive variance
# compared to additive variance"
SP <- AlphaSimR::SimParam$new(founderHaps)
relAA <- 0.5
SP$addTraitADE(nQtlPerChr=nQTL, meanDD=meanDD, varDD=varDD, relAA=relAA)

# Create a new population of founders
founders <- AlphaSimR::newPop(founderHaps, simParam=SP)

# Compare genotypic and breeding values
corBVGV <- cor(AlphaSimR::gv(founders), AlphaSimR::bv(founders)) %>% round(3)
plot(AlphaSimR::gv(founders), AlphaSimR::bv(founders), pch=16,
     xlab="Genotypic value", ylab="Breeding value",
     main=paste0("BV vs GV: ADE model, cor=", corBVGV),
     cex.axis=1.3, cex.lab=1.3)


## Estimated versus analytical breeding value  
Estimating the breeding value by a progeny test.  
In practice, the BV can't be observed.  A progeny test calculates the average
deviation from the population mean of progeny from randomly mating an individual
to the population: 2 * mean(progeny pheno - pop mean)

In [ ]:
# Error variance for phenotypic evaluations
varE <- 1
# Number of progeny for breeding value estimation
nProgeny1 <- 5
nProgeny2 <- 50

# Estimate breeding values
# ind is the individual whose breeding value you want to estimate
# pop is the population that individual is in
# nProgeny is the number of progeny for the test
# varE is the error variance with which phenotypes are evaluated
#      if the genotypic variance is 1 then varE=1 will give h2 = 0.5
estimateBV <- function(ind, pop, nProgeny, varE=1){
  # I'm going to cheat a little and assume we know the population mean exactly
  popMean <- AlphaSimR::gv(pop) %>% mean
  # Set up crossPlan to cross ind to random others nProgeny times
  crossPlan <- cbind(ind, sample(AlphaSimR::nInd(pop), nProgeny, replace=T))
  progeny <- AlphaSimR::makeCross(pop, crossPlan)
  progPheno <- AlphaSimR::setPheno(progeny, varE=varE, onlyPheno=T)
  return(2*(mean(progPheno) - popMean))
}

# estimate BV with a progeny test of nProgeny1
estimatedBV <- sapply(1:AlphaSimR::nInd(founders), estimateBV,
                      pop=founders, nProgeny=nProgeny1, varE=varE)
# Compare estimated and analytical breeding values
plot(estimatedBV, AlphaSimR::bv(founders), pch=16,
     xlab="Estimated value", ylab="Analytical value",
     main=paste("Breeding value estimated from", nProgeny1, "Progeny"))

# estimate BV with a progeny test of nProgeny2
estimatedBV <- sapply(1:AlphaSimR::nInd(founders), estimateBV,
                      pop=founders, nProgeny=nProgeny2, varE=varE)
# Compare estimated and analytical breeding values
plot(estimatedBV, AlphaSimR::bv(founders), pch=16,
     xlab="Estimated value", ylab="Analytical value",
     main=paste("Breeding value estimated from", nProgeny2, "Progeny"))

## Round robin crossing design  
This is a simple design that ensures that all parents get used equally  

In [ ]:
makeRoundRobin <- function(pop, makeRandom=F){
  nInd <- AlphaSimR::nInd(pop)
  parOrder <- 1:nInd
  if (makeRandom) parOrder <- sample(parOrder)
  return(cbind(parOrder, parOrder[c(2:nInd, 1)]))
}

# Make a bunch of new lines
crossPlan <- makeRoundRobin(founders)
exptLines <- AlphaSimR::makeCross(founders, crossPlan)

cbind(exptLines@id, exptLines@mother, exptLines@father) %>% head

## `records` data structure  
`AlphaSimR` populations only retain phenotypes of the most recent evaluation.
In plant breeding, it is common to evaluate the same line more than once,
and it makes sense to include all of those phenotypes in downstream analyses.
Here, I propose a simple tibble to retain phenotypic records.  

### Set up records
The columns are the individual `id`, its seed parent id `seedPar`, its pollen
parent id `pollenPar`, its observed phenotype `pheno`, and the error variance
of that observed phenotype `varE`  

In [ ]:
nInd <- 100
varE <- 1
# Function to make a simple data structure out of a population
# AlphaSimR doesn't retain varE once you have setPheno, so supply it
makeRecFromPop <- function(pop, varE=1){
  dplyr::tibble(id=pop@id,
                seedPar=pop@mother,
                pollenPar=pop@father,
                pheno=AlphaSimR::pheno(pop),
                varE=varE)
}

# Run a phenotyping experiment and populate the `records` data structure
exptLines <- AlphaSimR::setPheno(exptLines, varE=varE, simParam=SP)
records <- makeRecFromPop(exptLines, varE=varE)
head(records)

# Compare genotypic and phenotypic values
corPhGV <- cor(AlphaSimR::gv(exptLines), AlphaSimR::pheno(exptLines)) %>%
  round(3)
plot(AlphaSimR::gv(exptLines), AlphaSimR::pheno(exptLines), pch=16,
     xlab="Genetic value", ylab="Phenotype",
     main=paste0("First Evaluation, cor=", corPhGV))

# Second evaluation
exptLines <- AlphaSimR::setPheno(exptLines, varE=varE, simParam=SP)
# Add the new records to the old ones
records <- dplyr::bind_rows(records, makeRecFromPop(exptLines, varE=varE))

# Compare genotypic and phenotypic values
corPhGV <- cor(AlphaSimR::gv(exptLines), AlphaSimR::pheno(exptLines)) %>%
  round(3)
plot(AlphaSimR::gv(exptLines), AlphaSimR::pheno(exptLines), pch=16,
     xlab="Genetic value", ylab="Phenotype",
     main=paste0("Second Evaluation, cor=", corPhGV))

## Use records to estimate genotypic value from >1 observation

In [ ]:
# BASIC LINEAR MODEL
lmEstGV <- lm(pheno ~ -1 + id, data=records)
gvEstimates <- coefficients(lmEstGV)

# Compare genotypic and **Combined** phenotypic values
corEstGV <- cor(AlphaSimR::gv(exptLines), gvEstimates) %>%
  round(3)
plot(AlphaSimR::gv(exptLines), gvEstimates, pch=16,
     xlab="Genetic value", ylab="Phenotype",
     main=paste0("Both Evaluations, cor=", corEstGV))

## Use records in two-stage selection
In plant breeding there is often a preliminary evaluation stage, then a
more advanced stage.  We can use the records to keep track of these stages.

In [ ]:
nIndStage1 <- 400
nIndStage2 <- 100
varEstage1 <- 4
varEstage2 <- 1

progenyPerCross <- nIndStage1 / nrow(crossPlan)
# The production pipeline starts with a bunch of new lines
exptLines <- AlphaSimR::makeCross(founders, crossPlan,
                                  nProgeny=progenyPerCross)

# Phenotypic evaluation of experimental lines
exptLines <- AlphaSimR::setPheno(exptLines, varE=varEstage1, simParam=SP)
records <- makeRecFromPop(exptLines, varE=varEstage1)

# Select among lines to advance to Stage 2
keep <- AlphaSimR::pheno(exptLines) %>%
  order(decreasing=T) %>% .[1:nIndStage2]

# Phenotypic evaluation of Stage 2 lines
stage2Lines <- exptLines[keep]
stage2Lines <- AlphaSimR::setPheno(stage2Lines, varE=varEstage2, simParam=SP)
records <- dplyr::bind_rows(records, makeRecFromPop(stage2Lines,
                                                    varE=varEstage2))

str(records)

## Estimate gain from selection four ways
This little section is mostly to illustrate that `AlphaSimR` thinks of
breeding values *relative to the population mean*, but of genotypic values in
*absolute* terms.

In [ ]:
print("Gain from selection on BV where the reference pop is stage2Lines")
print(paste("Gain from selection",
            round(mean(AlphaSimR::bv(stage2Lines)) -
                  mean(AlphaSimR::bv(exptLines)), 2)))

print("Gain from selection on BV where the reference pop is all exptLines")
print(paste("Gain from selection",
            round(mean(AlphaSimR::bv(exptLines)[keep]) -
                  mean(AlphaSimR::bv(exptLines)), 2)))

print("Gain from selection on **GV** where the reference pop is stage2Lines")
print(paste("Gain from selection",
            round(mean(AlphaSimR::gv(stage2Lines)) -
                  mean(AlphaSimR::gv(exptLines)), 2)))

print("Gain from selection on **GV** where the reference pop is all exptLines")
print(paste("Gain from selection",
            round(mean(AlphaSimR::gv(exptLines)[keep]) -
                  mean(AlphaSimR::gv(exptLines)), 2)))

## Homework  
1. (**3 pts**) We saw how to write a function to estimate breeding values by doing a progeny
test (chunk `Estimate breeding value`). Then, in the chunk `Linear model
estimation`, we used a simple linear model to combine records to estimate the
genotypic values of individuals (chunk `Linear model estimation`).  
Tweak the linear model so that it estimates, for example, the breeding
values of the seed parent. HINT: this will entail introducing `seedPar`
into the model. Use the `exptLines` and their associated `records` from the
`Two stage selection` chunk.  
2. (**2 pts**) Make plots comparing the breeding values thus obtained to those
obtained by the `bv` function or by our own `estimateBV` function.  
3. (**3 pts**) The `lm` function from the `stats` package can account for observations with
different weights. Weights should be the inverse of the error variance for the
observation. Handily, we keep that error variance in the `records` tibble.
Again, tweak the linear model and use the records from the `Two stage selection`
chunk to account for the different error variances from Stage 1 versus Stage 2
when you estimate the genotypic values for individuals.  
4. (**2 pts**) Correlate the genotypic values obtained with or without weighting to the
simulated values obtained by the `AlphaSimR::gv` function.  